In [1]:
import os

os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [2]:
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
import glob
from IPython.core.display_functions import clear_output

In [5]:
import sys  
sys.path.insert(1, '../../../')


import experiment, autoencoders

### Parameters

In [14]:
num_endmembers = 5

low_snr_epochs = 15
high_snr_epochs = 50

model_replicates = 5
experiment_seed = 42

results_folder = '../results'
raw_data_folder = '../raw datasets'

In [15]:
datasets = sorted(glob.glob(os.path.join(raw_data_folder, '*')))
datasets

['../raw datasets/High SNR',
 '../raw datasets/High SNR (no refs)',
 '../raw datasets/Low SNR',
 '../raw datasets/Low SNR (no refs)']

In [8]:
def display_results(metrics):
    results_order = ['PCA', 'N-FINDR + FCLS', 'VCA + FCLS', 'Dense AE', 'Convolutional AE', 'Transformer AE', 'Convolutional Transformer AE']

    metrics['SAD'] = metrics[['SAD_mean', 'SAD_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)
    metrics['PCC'] = metrics[['PCC_mean', 'PCC_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)
    metrics['MSE'] = metrics[['MSE_mean', 'MSE_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)

    display(metrics.set_index('index').reindex(results_order)[['SAD', 'PCC', 'MSE']])

In [9]:
results = {}

### +preprocessing scenario

In [16]:
results_subfolder = 'preprocessing_sad'
result_folder = os.path.join(results_folder, results_subfolder)

results[results_subfolder] = {}
for dataset in datasets:
    custom_epochs = low_snr_epochs if 'Low' in dataset else high_snr_epochs
    
    # Run experiment
    metrics = experiment.run([dataset], num_endmembers, model_replicates=model_replicates, epochs=custom_epochs, experiment_folder=result_folder, seed=experiment_seed, preprocess=True, ae_loss=autoencoders.SAD)

    results[results_subfolder][dataset] = metrics

clear_output()

for scene, metrics in results[results_subfolder].items():
    print("Results for scene: ", scene)
    display_results(metrics)

Results for scene:  ../raw datasets/High SNR


,SAD,PCC,MSE
index,,,
PCA,1.018 ± 0.0,0.547 ± 0.0,0.073 ± 0.0
N-FINDR + FCLS,0.202 ± 0.0,0.039 ± 0.0,0.005 ± 0.0
VCA + FCLS,0.322 ± 0.044,0.099 ± 0.047,0.004 ± 0.0
Dense AE,0.212 ± 0.004,0.029 ± 0.001,0.057 ± 0.001
Convolutional AE,0.203 ± 0.001,0.027 ± 0.0,0.058 ± 0.001
Transformer AE,0.206 ± 0.001,0.028 ± 0.0,0.058 ± 0.001
Convolutional Transformer AE,0.208 ± 0.002,0.028 ± 0.0,0.057 ± 0.001


Results for scene:  ../raw datasets/High SNR (no refs)


,SAD,PCC,MSE
index,,,
PCA,1.026 ± 0.0,0.541 ± 0.0,0.072 ± 0.0
N-FINDR + FCLS,0.515 ± 0.035,0.203 ± 0.055,0.037 ± 0.029
VCA + FCLS,0.525 ± 0.026,0.221 ± 0.056,0.042 ± 0.029
Dense AE,0.219 ± 0.001,0.031 ± 0.0,0.058 ± 0.001
Convolutional AE,0.215 ± 0.002,0.03 ± 0.001,0.058 ± 0.001
Transformer AE,0.218 ± 0.002,0.031 ± 0.0,0.058 ± 0.002
Convolutional Transformer AE,0.218 ± 0.001,0.031 ± 0.0,0.057 ± 0.001


Results for scene:  ../raw datasets/Low SNR


,SAD,PCC,MSE
index,,,
PCA,1.173 ± 0.0,0.641 ± 0.0,0.075 ± 0.0
N-FINDR + FCLS,0.9 ± 0.0,0.52 ± 0.0,0.026 ± 0.0
VCA + FCLS,0.677 ± 0.022,0.325 ± 0.019,0.077 ± 0.035
Dense AE,0.462 ± 0.038,0.164 ± 0.042,0.058 ± 0.005
Convolutional AE,0.503 ± 0.055,0.207 ± 0.052,0.058 ± 0.002
Transformer AE,0.496 ± 0.05,0.2 ± 0.048,0.061 ± 0.002
Convolutional Transformer AE,0.521 ± 0.057,0.226 ± 0.059,0.068 ± 0.013


Results for scene:  ../raw datasets/Low SNR (no refs)


,SAD,PCC,MSE
index,,,
PCA,1.21 ± 0.0,0.691 ± 0.0,0.073 ± 0.0
N-FINDR + FCLS,1.077 ± 0.0,0.641 ± 0.0,0.154 ± 0.0
VCA + FCLS,0.901 ± 0.018,0.482 ± 0.005,0.108 ± 0.006
Dense AE,0.503 ± 0.046,0.198 ± 0.046,0.054 ± 0.007
Convolutional AE,0.529 ± 0.041,0.228 ± 0.03,0.057 ± 0.003
Transformer AE,0.545 ± 0.037,0.243 ± 0.032,0.069 ± 0.013
Convolutional Transformer AE,0.533 ± 0.035,0.236 ± 0.034,0.062 ± 0.011


In [18]:
results_subfolder = 'preprocessing_mse+sad'
result_folder = os.path.join(results_folder, results_subfolder)

results[results_subfolder] = {}
for dataset in datasets:
    custom_epochs = low_snr_epochs if 'Low' in dataset else high_snr_epochs
    
    # Run experiment
    metrics = experiment.run([dataset], num_endmembers, model_replicates=model_replicates, epochs=custom_epochs, experiment_folder=result_folder, seed=experiment_seed, preprocess=True, ae_loss=autoencoders.MSE_SAD(1e3))

    results[results_subfolder][dataset] = metrics

clear_output()

for scene, metrics in results[results_subfolder].items():
    print("Results for scene: ", scene)
    display_results(metrics)

Results for scene:  ../raw datasets/High SNR


,SAD,PCC,MSE
index,,,
PCA,1.018 ± 0.0,0.547 ± 0.0,0.073 ± 0.0
N-FINDR + FCLS,0.202 ± 0.0,0.039 ± 0.0,0.005 ± 0.0
VCA + FCLS,0.322 ± 0.044,0.099 ± 0.047,0.004 ± 0.0
Dense AE,0.242 ± 0.008,0.037 ± 0.002,0.003 ± 0.001
Convolutional AE,0.264 ± 0.068,0.056 ± 0.044,0.004 ± 0.004
Transformer AE,0.234 ± 0.006,0.035 ± 0.002,0.004 ± 0.001
Convolutional Transformer AE,0.321 ± 0.121,0.104 ± 0.096,0.008 ± 0.005


Results for scene:  ../raw datasets/High SNR (no refs)


,SAD,PCC,MSE
index,,,
PCA,1.026 ± 0.0,0.541 ± 0.0,0.072 ± 0.0
N-FINDR + FCLS,0.515 ± 0.035,0.203 ± 0.055,0.037 ± 0.029
VCA + FCLS,0.525 ± 0.026,0.221 ± 0.056,0.042 ± 0.029
Dense AE,0.265 ± 0.04,0.047 ± 0.018,0.003 ± 0.002
Convolutional AE,0.24 ± 0.007,0.037 ± 0.002,0.002 ± 0.0
Transformer AE,0.24 ± 0.009,0.037 ± 0.002,0.004 ± 0.001
Convolutional Transformer AE,0.333 ± 0.123,0.108 ± 0.099,0.007 ± 0.006


Results for scene:  ../raw datasets/Low SNR


,SAD,PCC,MSE
index,,,
PCA,1.173 ± 0.0,0.641 ± 0.0,0.075 ± 0.0
N-FINDR + FCLS,0.9 ± 0.0,0.52 ± 0.0,0.026 ± 0.0
VCA + FCLS,0.677 ± 0.022,0.325 ± 0.019,0.077 ± 0.035
Dense AE,0.498 ± 0.038,0.174 ± 0.032,0.07 ± 0.006
Convolutional AE,0.568 ± 0.057,0.243 ± 0.044,0.083 ± 0.017
Transformer AE,0.606 ± 0.034,0.296 ± 0.041,0.101 ± 0.026
Convolutional Transformer AE,0.57 ± 0.068,0.25 ± 0.072,0.113 ± 0.012


Results for scene:  ../raw datasets/Low SNR (no refs)


,SAD,PCC,MSE
index,,,
PCA,1.21 ± 0.0,0.691 ± 0.0,0.073 ± 0.0
N-FINDR + FCLS,1.077 ± 0.0,0.641 ± 0.0,0.154 ± 0.0
VCA + FCLS,0.901 ± 0.018,0.482 ± 0.005,0.108 ± 0.006
Dense AE,0.509 ± 0.045,0.185 ± 0.042,0.069 ± 0.008
Convolutional AE,0.593 ± 0.052,0.272 ± 0.04,0.08 ± 0.013
Transformer AE,0.632 ± 0.014,0.311 ± 0.02,0.097 ± 0.028
Convolutional Transformer AE,0.665 ± 0.179,0.348 ± 0.176,0.164 ± 0.037


In [26]:
results_subfolder = 'preprocessing_customised'
result_folder = os.path.join(results_folder, results_subfolder)

results[results_subfolder] = {}
for dataset in datasets:
    custom_epochs = low_snr_epochs if 'Low' in dataset else high_snr_epochs
    custom_ae_loss = autoencoders.SAD if 'Low' in dataset else autoencoders.MSE_SAD(1e3)

    # Run experiment
    metrics = experiment.run([dataset], num_endmembers, model_replicates=model_replicates, epochs=custom_epochs, experiment_folder=result_folder, seed=experiment_seed, preprocess=True, ae_loss=custom_ae_loss)

    results[results_subfolder][dataset] = metrics

clear_output()

for scene, metrics in results[results_subfolder].items():
    print("Results for scene: ", scene)
    display_results(metrics)

Results for scene:  raw data/High SNR


,SAD,PCC,MSE
index,,,
PCA,1.018 ± 0.0,0.547 ± 0.0,0.073 ± 0.0
N-FINDR + FCLS,0.202 ± 0.0,0.039 ± 0.0,0.005 ± 0.0
VCA + FCLS,0.322 ± 0.044,0.099 ± 0.047,0.004 ± 0.0
Dense AE,0.242 ± 0.008,0.037 ± 0.002,0.003 ± 0.001
Convolutional AE,0.264 ± 0.068,0.056 ± 0.044,0.004 ± 0.004
Transformer AE,0.234 ± 0.006,0.035 ± 0.002,0.004 ± 0.001
Convolutional Transformer AE,0.321 ± 0.121,0.104 ± 0.096,0.008 ± 0.005


Results for scene:  raw data/High SNR (no refs)


,SAD,PCC,MSE
index,,,
PCA,1.026 ± 0.0,0.541 ± 0.0,0.072 ± 0.0
N-FINDR + FCLS,0.515 ± 0.035,0.203 ± 0.055,0.037 ± 0.029
VCA + FCLS,0.525 ± 0.026,0.221 ± 0.056,0.042 ± 0.029
Dense AE,0.265 ± 0.04,0.047 ± 0.018,0.003 ± 0.002
Convolutional AE,0.24 ± 0.007,0.037 ± 0.002,0.002 ± 0.0
Transformer AE,0.24 ± 0.009,0.037 ± 0.002,0.004 ± 0.001
Convolutional Transformer AE,0.333 ± 0.123,0.108 ± 0.099,0.007 ± 0.006


Results for scene:  raw data/Low SNR


,SAD,PCC,MSE
index,,,
PCA,1.173 ± 0.0,0.641 ± 0.0,0.075 ± 0.0
N-FINDR + FCLS,0.9 ± 0.0,0.52 ± 0.0,0.026 ± 0.0
VCA + FCLS,0.677 ± 0.022,0.325 ± 0.019,0.077 ± 0.035
Dense AE,0.462 ± 0.038,0.164 ± 0.042,0.058 ± 0.005
Convolutional AE,0.503 ± 0.055,0.207 ± 0.052,0.058 ± 0.002
Transformer AE,0.496 ± 0.05,0.2 ± 0.048,0.061 ± 0.002
Convolutional Transformer AE,0.521 ± 0.057,0.226 ± 0.059,0.068 ± 0.013


Results for scene:  raw data/Low SNR (no refs)


,SAD,PCC,MSE
index,,,
PCA,1.21 ± 0.0,0.691 ± 0.0,0.073 ± 0.0
N-FINDR + FCLS,1.077 ± 0.0,0.641 ± 0.0,0.154 ± 0.0
VCA + FCLS,0.901 ± 0.018,0.482 ± 0.005,0.108 ± 0.006
Dense AE,0.503 ± 0.046,0.198 ± 0.046,0.054 ± 0.007
Convolutional AE,0.529 ± 0.041,0.228 ± 0.03,0.057 ± 0.003
Transformer AE,0.545 ± 0.037,0.243 ± 0.032,0.069 ± 0.013
Convolutional Transformer AE,0.533 ± 0.035,0.236 ± 0.034,0.062 ± 0.011
